# Evaluation script

## Load packages

In [1]:
Sys.setenv("PKG_CXXFLAGS"="-std=c++14")

# List of required packages
list_of_packages <- c("here", "dplyr")

# Function to load packages and handle errors
load_if_installed <- function(p) {
  tryCatch({
    library(p, character.only = TRUE)
  }, error = function(e) {
    message(sprintf("Package '%s' is not installed.", p))
  })
}

# Load all required packages
lapply(list_of_packages, load_if_installed)

here() starts at /home/uni08/hpc/emma.foessing01/u11969/Master-Thesis




Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[[1]]
[1] "here"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
[1] "dplyr"     "here"      "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"

## Load results

In [ ]:
datanames <- c("cps", "adult")
modelnames <- c("orig", "cart", "rf", "bn", "xgb", "svm")

loaded_data <- list()

# Loop through all combinations of dataname and modelname
for (dataname in datanames) {
  for (modelname in modelnames) {
    # Construct the file path
    file_path <- paste0(here(), "/results/", dataname, "_", modelname, "_res.RData")
    
    # Check if the file exists
    if (file.exists(file_path)) {
      # Load the file if it exists
      load(file = file_path)
      
      # Assign a names
      loaded_data[[paste0(dataname, "_", modelname)]] <- get(ls()[1])
      
      cat("Loaded data for:", dataname, modelname, "\n")
    } else {
      # Print message if the file does not exist
      cat("Data does not exist for:", dataname, modelname, "\n")
    }
  }
}

# Print all loaded dataframes
cat("\n--- List of loaded dataframes ---\n")
for (name in names(loaded_data)) {
  cat("Data for:", name, "\n")
  print(loaded_data[[name]])
}

In [ ]:
# Initialize lists to store predictions for each metric for cps and adult
cps_metric_tables <- list(
  MAE = list(),
  MSE = list(),
  RMSE = list(),
  R_squared = list(),
  MAPE = list()
)

adult_metric_tables <- list(
  Accuracy = list(),
  F1 = list(),
  Sensitivity = list(),
  Specificity = list()
)

# Loop through the loaded data and extract the metrics for each model
for (name in names(loaded_data)) {
  dataset_name <- sub("_(cart|rf|bn|xgb|svm|orig)", "", name)
  
  # Check if the data belongs to cps or adult
  if (grepl("cps", name)) {
    # Check for regression metrics in eval_regression_avg
    if ("eval_regression_avg" %in% names(loaded_data[[name]])) {
      cps_metric_tables$MAE[[dataset_name]] <- loaded_data[[name]]$eval_regression_avg["CART", "MAE"]
      cps_metric_tables$MSE[[dataset_name]] <- loaded_data[[name]]$eval_regression_avg["CART", "MSE"]
      cps_metric_tables$RMSE[[dataset_name]] <- loaded_data[[name]]$eval_regression_avg["CART", "RMSE"]
      cps_metric_tables$R_squared[[dataset_name]] <- loaded_data[[name]]$eval_regression_avg["CART", "R_squared"]
      cps_metric_tables$MAPE[[dataset_name]] <- loaded_data[[name]]$eval_regression_avg["CART", "MAPE"]
    }
  } else if (grepl("adult", name)) {
    # Check for classification metrics in eval_classification_avg
    if ("eval_classification_avg" %in% names(loaded_data[[name]])) {
      adult_metric_tables$Accuracy[[dataset_name]] <- loaded_data[[name]]$eval_classification_avg["CART", "Accuracy"]
      adult_metric_tables$F1[[dataset_name]] <- loaded_data[[name]]$eval_classification_avg["CART", "F1"]
      adult_metric_tables$Sensitivity[[dataset_name]] <- loaded_data[[name]]$eval_classification_avg["CART", "Sensitivity"]
      adult_metric_tables$Specificity[[dataset_name]] <- loaded_data[[name]]$eval_classification_avg["CART", "Specificity"]
    }
  }
}

# Convert the lists to dataframes for easier viewing and processing
for (metric in names(cps_metric_tables)) {
  cps_metric_tables[[metric]] <- as.data.frame(do.call(rbind, cps_metric_tables[[metric]]))
  rownames(cps_metric_tables[[metric]]) <- paste0(names(cps_metric_tables[[metric]]), "_cps")
}

for (metric in names(adult_metric_tables)) {
  adult_metric_tables[[metric]] <- as.data.frame(do.call(rbind, adult_metric_tables[[metric]]))
  rownames(adult_metric_tables[[metric]]) <- paste0(names(adult_metric_tables[[metric]]), "_adult")
}

# Print the resulting metric tables
cat("\n--- CPS Metric Tables ---\n")
for (metric in names(cps_metric_tables)) {
  cat("\nMetric:", metric, "\n")
  print(cps_metric_tables[[metric]])
}

cat("\n--- Adult Metric Tables ---\n")
for (metric in names(adult_metric_tables)) {
  cat("\nMetric:", metric, "\n")
  print(adult_metric_tables[[metric]])
}
